# File Input
Takes in a text file and returns a vector of ordered unigrams U. 
It should also consider stemming and other relevant pre-processing. Josh's note: parse "African American" as a unigram.

# Correlation Matrix
Parse U to create a word-word frequency matrix M, where each row represents a word and each entry xi,j represents the number of times word i co-occurs with word j.
Convert M to a new matrix M’ with some sort of correlation operation. We could use PMI, Occai (see Josh’s paper), CSA, or some other correlation structure.
Let row a represent the unigram “African American”. Take in that row, and output an ordered list of (this_unigram, correlation_score) pairs which represent the correlation score of this_unigram with the term “African American”
Produce a list L of the top 100 correlated words with the term “African American”
 


# Sentiment Analysis
Takes in a vector V of words and returns the average sentiment score across all terms in V as determined by freebase. Note to Jason: consider other sentiment databases

# XOR/AND
Takes in a dict of corpus:list of words and returns a dict of corpus:XOR words and dict of corpus:AND words.

# Word Cloud
Takes in a 